In [4]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import glob

file = r'E:\safety_sim\dataset\범죄안전데이터\서울시 경찰청별 범죄 발생 및 검거 현황\불법주정차 신고현황'
file_list = glob.glob(f"{file}/*.csv")

dataframes =[]
for data in file_list:
    df = pd.read_csv(data)
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)

In [5]:
df

,민원접수일,민원접수시간,주소,경도,위도
0,20210929,00:05:00,서울 강서구 화곡로66길 130,126.859911,37.555537
1,20210929,00:38:00,중구 다산로48길 33 단우물어린이집앞,127.018013,37.567483
2,20210929,00:38:00,서울특별시 중구 마장로 19,127.012793,37.569183
3,20210929,00:50:00,서울특별시 강서구 까치산로15길 21,126.847125,37.547040
4,20210929,01:15:00,서울특별시 중구 다산로 293,127.015703,37.569085
...,...,...,...,...,...
3070523,20240313,23:55:00,(신림동) 남부순환로 1619-1,126.930651,37.484679
3070524,20240313,23:55:00,신사동 554-23/ 앞 이면도로 주변,127.024768,37.521643
3070525,20240313,23:56:00,서울 동대문구 휘경동 183-127,127.056992,37.591036
3070526,20240313,23:56:00,서울특별시 마포구 희우정로20길 71,126.904288,37.556222


In [7]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, Point, Polygon, MultiPolygon
import json



file_path = 'E:\safety_sim\dataset\서울특별시.json'

# GeoJSON 파일 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# MultiPolygon을 개별 Polygon으로 처리
flattened_geometries = []
adm_names = []

# GeoJSON에서 MultiPolygon을 Polygon으로 나누어 저장
for feature in geojson_data['features']:
    geom = shape(feature['geometry'])
    if isinstance(geom, MultiPolygon):
        # MultiPolygon 객체에서 개별 Polygon을 추출
        for polygon in geom.geoms:
            flattened_geometries.append(polygon)
            adm_names.append(feature['properties']['adm_nm'])
    elif isinstance(geom, Polygon):
        flattened_geometries.append(geom)
        adm_names.append(feature['properties']['adm_nm'])

# GeoDataFrame 생성 (Flattened된 다각형 처리)
gdf_flat = gpd.GeoDataFrame(
    {'adm_nm': adm_names, 'geometry': flattened_geometries},
    crs="EPSG:4326"
)

# CSV 데이터를 GeoDataFrame으로 변환
csv_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['경도'], df['위도']), crs="EPSG:4326")

# 위도, 경도에 해당하는 구 찾기 위한 공간 조인 수행
csv_gdf = gpd.sjoin(csv_gdf, gdf_flat[['adm_nm', 'geometry']], how="left", predicate='within')

# 결과 출력
print(csv_gdf[['민원접수일', '민원접수시간', '주소', '경도', '위도', 'adm_nm']].head())

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\hyunj\AppData\Local\Temp\ipykernel_20136\650782451.py:8: SyntaxWarning: invalid escape sequence '\s'
  file_path = 'E:\safety_sim\dataset\서울특별시.json'


      민원접수일    민원접수시간                     주소          경도         위도  \
0  20210929  00:05:00      서울 강서구 화곡로66길 130  126.859911  37.555537   
1  20210929  00:38:00  중구 다산로48길 33 단우물어린이집앞  127.018013  37.567483   
2  20210929  00:38:00        서울특별시 중구 마장로 19  127.012793  37.569183   
3  20210929  00:50:00   서울특별시 강서구 까치산로15길 21  126.847125  37.547040   
4  20210929  01:15:00       서울특별시 중구 다산로 293  127.015703  37.569085   

           adm_nm  
0  서울특별시 강서구 등촌1동  
1    서울특별시 중구 황학동  
2    서울특별시 중구 신당동  
3  서울특별시 강서구 화곡본동  
4    서울특별시 중구 신당동  


In [9]:
csv_gdf['자치구'] = csv_gdf['adm_nm'].str.extract(r'서울특별시\s(.*?)\s')
df_counts = csv_gdf['자치구'].value_counts().reset_index()
df_counts.columns = ['자치구', '개수']

df_counts


,자치구,개수
0,강남구,319427
1,마포구,285247
2,강서구,228511
3,서초구,210348
4,용산구,182666
5,관악구,178689
6,송파구,161575
7,영등포구,143313
8,중구,138668
9,강동구,125536


In [10]:
import json
import folium
import pandas as pd
from branca.colormap import LinearColormap

# 색상 단계 정의
colors = ['red', 'lightcoral', 'orange', 'blue', 'lightblue']
# 색상 범위 수동 조정
color_scale = LinearColormap(colors=colors,
                             vmin=df_counts['개수'].quantile(0.1),  # 10th percentile로 범위 설정
                             vmax=df_counts['개수'].quantile(0.9))  # 90th percentile로 범위 설정


# GeoJSON 파일 로드
geojson_path = r'E:\safety_sim\dataset\서울특별시.json'
with open(geojson_path, encoding='utf-8') as f:
    seoul_geo = json.load(f)

# GeoJSON에 'gu' 필드 추가
for feature in seoul_geo['features']:
    adm_nm = feature['properties']['adm_nm']  # 예: '서울특별시 성동구 행당동'
    gu_name = adm_nm.split()[1]  # '성동구'
    feature['properties']['gu'] = gu_name

# CCTV 데이터에서 자치구 필터링
geo_gu_list = {f['properties']['gu'] for f in seoul_geo['features']}
gu_counts_filtered = df_counts[df_counts['자치구'].isin(geo_gu_list)]

# folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 스타일 함수 정의 (색상 적용)
def style_function(feature):
    gu_name = feature['properties']['gu']
    value = gu_counts_filtered.loc[gu_counts_filtered['자치구'] == gu_name, '개수']
    if not value.empty:
        fill_color = color_scale(value.values[0])
    else:
        fill_color = 'gray'  # 데이터 없을 때
    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# GeoJson으로 색상 직접 입히기
folium.GeoJson(
    seoul_geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['gu'], aliases=['자치구'])
).add_to(m)

# 색상 범례 추가
color_scale.caption = '서울시 주차단속 수'
color_scale.add_to(m)

# 지도 저장
m.save("seoul_colors_car_crime.html")
